In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
import mlflow
import mlflow.sklearn
from sklearn.compose import make_column_selector as selector

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'mlflow'

In [7]:
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

train_df.dropna(subset=['SalePrice'], inplace=True)

X = train_df.drop(columns=['SalePrice', 'Id'])
y = train_df['SalePrice']

# Cleaning:

In [10]:
class DropHighNaN(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold
        self.to_drop = []

    def fit(self, X, y=None):
        self.to_drop = X.columns[X.isnull().mean() > self.threshold].tolist()
        return self

    def transform(self, X):
        return X.drop(columns=self.to_drop)

# Feature Engineering:

In [13]:
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, selector(dtype_include=['int64', 'float64'])),
    ('cat', categorical_pipeline, selector(dtype_include=['object']))
])

NameError: name 'selector' is not defined

# Feature Selection:

# Training:

In [ ]:
full_pipeline = Pipeline(steps=[
    ('drop_high_nan', DropHighNaN()),
    ('preprocessing', preprocessor),
    ('regressor', Ridge())
])

param_grid = {
    'drop_high_nan__threshold': [0.1, 0.2, 0.5, 0.8],
    'regressor__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
}
grid_search = GridSearchCV(full_pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error')


# Logging:

In [ ]:
import dagshub
import mlflow

dagshub.init(repo_owner='ashar-22', repo_name='hw01ml', mlflow=True)

with mlflow.start_run():
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    
    y_pred = best_model.predict(X_valid)
    rmse = mean_squared_error(np.log1p(y_valid), np.log1p(y_pred), squared=False)

    mlflow.log_param("best_alpha", grid_search.best_params_['regressor__alpha'])
    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(best_model, "ridge_regressor_model")

    print(f"Best alpha: {grid_search.best_params_['regressor__alpha']}")
    print(f"Validation RMSE (log1p): {rmse:.4f}")